In [2]:
# from models.tokenizer import Tokenizer
from models.model import *

In [3]:
import torch
import os

In [4]:
# ModelArgs(dim=4096, n_layers=32, n_heads=32, n_kv_heads=None, vocab_size=32000, multiple_of=256, ffn_dim_multiplier=None, norm_eps=1e-05, rope_theta=500000, max_batch_size=32, max_seq_len=2048)
args = ModelArgs(
    vocab_size=32768,
    dim=512,
    n_layers=6,
    # dim=1024,
    # n_layers=16,
    # dim=2048,
    # n_layers=24,
    n_heads=16,
    ffn_dim_multiplier=2,
)
model = Transformer(args)
args

ModelArgs(dim=512, n_layers=6, n_heads=16, n_kv_heads=None, vocab_size=32768, multiple_of=1, ffn_dim_multiplier=2, norm_eps=1e-05, rope_theta=500000, max_batch_size=32, max_seq_len=2048)

In [5]:
total_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
# model.tok_embeddings.weight.nbytes
embedding_bytes = model.tok_embeddings.weight.numel() * model.tok_embeddings.weight.element_size()
model_bytes = total_bytes - embedding_bytes
# total_bytes, embedding_bytes,
# f'{total_bytes:,}', '175,138,816'

print(f'Model size:     {model_bytes:,} bytes')
print(f'Embedding size: {embedding_bytes:,} bytes')
print(f'Total size:     {total_bytes:,} bytes')
print()

total_params = sum(p.numel() for p in model.parameters())
embedding_params = model.tok_embeddings.weight.numel()
model_params = total_params - embedding_params

print(f'Model params:     {model_params:,}')
print(f'Embedding params: {embedding_params:,}')
print(f'Total params:     {total_params:,}')

Model size:     130,050,048 bytes
Embedding size: 67,108,864 bytes
Total size:     197,158,912 bytes

Model params:     32,512,512
Embedding params: 16,777,216
Total params:     49,289,728


In [ ]:
param_dict = {pn: p for pn, p in model.named_parameters()}
param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]

print(f'Decay params:   {sum(p.numel() for p in decay_params):13,}')
print(f'Nodecay params: {sum(p.numel() for p in nodecay_params):13,}')

Decay params:      49,283,072
Nodecay params:         6,656


In [8]:
for n, p in param_dict.items():
    if p.dim() < 2:
            print(f'{n:80} {p.numel():13,}')

layers.0.attention_norm.weight                                                           2,048
layers.0.ffn_norm.weight                                                                 2,048
layers.1.attention_norm.weight                                                           2,048
layers.1.ffn_norm.weight                                                                 2,048
layers.2.attention_norm.weight                                                           2,048
layers.2.ffn_norm.weight                                                                 2,048
layers.3.attention_norm.weight                                                           2,048
layers.3.ffn_norm.weight                                                                 2,048
layers.4.attention_norm.weight                                                           2,048
layers.4.ffn_norm.weight                                                                 2,048
layers.5.attention_norm.weight                    

In [6]:
model(torch.randint(0, 32000, (2, 512)), start_pos=0).shape

torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
torch.Size([2, 512, 1024]) tensor(0)
t

torch.Size([2, 512, 32768])

In [7]:
model.layers[0].feed_forward

FeedForward(
  (w1): Linear(in_features=1024, out_features=2048, bias=False)
  (w2): Linear(in_features=2048, out_features=1024, bias=False)
  (w3): Linear(in_features=1024, out_features=2048, bias=False)
)

# Del later

In [10]:
raise Exception

Exception: 

In [ ]:
import torch
import torch.nn as nn

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(512, 512)
        # self.t = torch.ones(512, 512)
        self.register_buffer('t', torch.ones(512, 512), persistent=False)
        
    def forward(self, x):
        x = self.l1(x)
        x = torch.matmul(x, self.t)
        return x

In [ ]:
model = Model()
print(model.t.device, model.l1.weight.device)
model = model.to('cuda:0')
print(model.t.device, model.l1.weight.device)
print(model.state_dict().keys())


cpu cpu
cuda:0 cuda:0
odict_keys(['l1.weight', 'l1.bias'])


In [ ]:
from transformers import AutoTokenizer

mistral

In [ ]:
# os.environ['RANK'] = '0'
# os.environ['WORLD_SIZE'] = '1'
# os.environ['MASTER_ADDR'] = 'localhost'
# os.environ['MASTER_PORT'] = '29500'
# torch.distributed.init_process_group()

# from fairscale.nn.model_parallel.initialize import initialize_model_parallel
# initialize_model_parallel(1)

In [1]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")

In [3]:
enc.n_vocab

50257